## 정규화

- 각 변수(데이터프레임의 열)에 들어 있는 숫자 데이터의 상대적 크기 차이 때문에 머신러닝 분석결과가 달라질 수 있다.
- 범위의 차이에 의해 범위크기 값이 큰 변수의 영향이 커지는 경향이 있다.
- 데이터값을 동일한 수준으로 맞춰 주어 중요도를 균일하게 하는 것이 정규화이다.
- 정규화를 거친 데이터의 범위는 0 ~ 1, -1 ~ 1 사이를 갖는다.


각 열의 데이터를 해당 열의 최대값으로 나누는 방법이 있다.

In [1]:
import pandas as pd
import numpy as np

In [24]:
import warnings
warnings.filterwarnings('ignore') # 각종 warning 무시하기

In [5]:
df = pd.read_csv('./auto-mpg.csv')

# horsepower 데이터 형태를 실수형을 변환 그전의 값 ? 삭제
df['horsepower'].replace('?', np.nan, inplace=True)
df.dropna(subset=['horsepower'], axis=0, inplace=True)
df['horsepower'] = df['horsepower'].astype('float')

# horsepower 열 최대값 확인
print(df['horsepower'].max())
print('\n')

df.horsepower = df.horsepower/abs(df.horsepower.max())
print(df.horsepower.head())

230.0


0    0.565217
1    0.717391
2    0.652174
3    0.652174
4    0.608696
Name: horsepower, dtype: float64


또한 각 열의 데이터 중에서 최대값과 최소값으로 뺀 값으로 나누는 방법이 있다. 각 열데이터에서 해당 열의 최소값을 뺀 값을 분자로 하고, 해당 열의 최대값과 최소값의 차를 분모로 하는 수를 계산하면 가장 큰값은 역시 1이 된다. 이것이 Min-Max 정규화이다.

In [6]:
df = pd.read_csv('./auto-mpg.csv')

# horsepower 열의 누락 데이터('?') 삭제하고 실수형으로 변환
df['horsepower'].replace('?', np.nan, inplace=True)      # '?'을 np.nan으로 변경
df.dropna(subset=['horsepower'], axis=0, inplace=True)   # 누락데이터 행을 삭제
df['horsepower'] = df['horsepower'].astype('float')      # 문자열을 실수형으로 변환

# horsepower 열의 통계 요약정보로 최대값(max)과 최소값(min)을 확인
print(df.horsepower.describe())
print('\n')

# horsepower 열의 최대값의 절대값으로 모든 데이터를 나눠서 저장
min_x = df.horsepower - df.horsepower.min()
min_max = df.horsepower.max() - df.horsepower.min()
df.horsepower = min_x / min_max

print(df.horsepower.head())
print('\n')
print(df.horsepower.describe())

count    392.000000
mean     104.469388
std       38.491160
min       46.000000
25%       75.000000
50%       93.500000
75%      126.000000
max      230.000000
Name: horsepower, dtype: float64


0    0.456522
1    0.646739
2    0.565217
3    0.565217
4    0.510870
Name: horsepower, dtype: float64


count    392.000000
mean       0.317768
std        0.209191
min        0.000000
25%        0.157609
50%        0.258152
75%        0.434783
max        1.000000
Name: horsepower, dtype: float64


## 시계열 데이터

- 주식, 환율 등 금융데이터를 다루기 위해 개발된 판다스는 시계열데이터(time series) 데이터를 다루는 여러 가지 유용한 기능을 제공한다. 
- 특시 시계열 데이터를 데이터프레임의 행 인덱스로 사용한다면 시간으로 기록된 데이터를 분석하는 것이 매우 편리하다.
- 판다스 시간 표현 방식 중에서 시계열 데이터 표현에 자주 사용되는 Timestamp, Period 가 있다. 말그대로 Timestamp 는 특정 시점, period 는 기간을 의미한다.

## 시계열 객체로 변환

- 우리가 접하는 많은 시간 데이터는 별도의 시간 자료형으로 기록되지 않고, 문자열 또는 숫자로 저장되는 경우가 많다. 이를 변환하는 함수를 알아볼 것이다.

### 문자열을 Timestamp로 변환

판다스에서 to_datetime() 함수를 사용하면 문자열 등 다른 자료형을 판다스 Timestamp를 나타내는 datatime64자료형으로 변환이 가능

예제 데이터는 주식 시장에서 거래되는 A종목의 거래 데이터를 정리한 csv파일을 활용한다.

In [9]:
df = pd.read_csv('./stock-data.csv')
# 데이터 내용 및 자료형 확인
print(df.head())
print('\n')
print(df.info())


         Date  Close  Start   High    Low  Volume
0  2018-07-02  10100  10850  10900  10000  137977
1  2018-06-29  10700  10550  10900   9990  170253
2  2018-06-28  10400  10900  10950  10150  155769
3  2018-06-27  10900  10800  11050  10500  133548
4  2018-06-26  10800  10900  11000  10700   63039


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB
None


Data열이 문자열(object)임을 알 수 있다. 이제 변환을 해보자,

In [11]:
df['New_Date'] = pd.to_datetime(df['Date'])
print(df.head())
print('\n')
print(df.info())
print('\n')
print(type(df['New_Date'][0]))

         Date  Close  Start   High    Low  Volume   New_Date
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      20 non-null     object        
 1   Close     20 non-null     int64         
 2   Start     20 non-null     int64         
 3   High      20 non-null     int64         
 4   Low       20 non-null     int64         
 5   Volume    20 non-null     int64         
 6   New_Date  20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 1.2+ KB
None


<class 'pandas._libs.tslibs.timestamps.T

In [12]:
df.set_index('New_Date', inplace=True)
df.drop('Date', axis=1, inplace=True)

# 데이터 내용 및 자료형 확인
print(df.head())
print('\n')
print(df.info())

            Close  Start   High    Low  Volume
New_Date                                      
2018-07-02  10100  10850  10900  10000  137977
2018-06-29  10700  10550  10900   9990  170253
2018-06-28  10400  10900  10950  10150  155769
2018-06-27  10900  10800  11050  10500  133548
2018-06-26  10800  10900  11000  10700   63039


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20 entries, 2018-07-02 to 2018-06-01
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Close   20 non-null     int64
 1   Start   20 non-null     int64
 2   High    20 non-null     int64
 3   Low     20 non-null     int64
 4   Volume  20 non-null     int64
dtypes: int64(5)
memory usage: 960.0 bytes
None


이렇게 시계열 값을 행인덱스로 지정하면 판다스는 DatetimeIndex로 저장해서 시계열 인덱스 클래스를 지원해 시간 순서에 맞춰 인덱스 또는 슬라이싱 하기 편리하다.

In [15]:
df.loc['2018-06-02':'2018-06-26']

/tmp/ipykernel_203/3399953645.py:1: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  df.loc['2018-06-02':'2018-06-26']


,Close,Start,High,Low,Volume
New_Date,,,,,
2018-06-26,10800,10900,11000,10700,63039
2018-06-25,11150,11400,11450,11000,55519
2018-06-22,11300,11250,11450,10750,134805
2018-06-21,11200,11350,11750,11200,133002
2018-06-20,11550,11200,11600,10900,308596
2018-06-19,11300,11850,11950,11300,180656
2018-06-18,12000,13400,13400,12000,309787
2018-06-15,13400,13600,13600,12900,201376
2018-06-14,13450,13200,13700,13150,347451


### Timestamp를 Period로 변환

- 판다스 to_period() 함수를 이용하면 일정한 기간을 나타내는  Period객체로 Timestamp 객체를 변환할 수 있다. freq 옵션에 기준이 되는 기간을 설정한다.
- 먼저 3개의 날짜 데이터를 Timestamp로 변환하고, 기간 옵션을 달리하여 Period 객체를 지정한다.
- freq 옵션에 'D'(1일의 기간), 'M'(1개월의 기간), 'A'(1년의 기간, 12월을 기준으로 삼는다) 이렇게 있다.

In [16]:
# 날짜 형식의 문자열로 구성되는 리스트 정의
dates = ['2019-01-01', '2020-03-01', '2021-06-01']

# 문자열 데이터(시리즈 객체)를 판다스 Timestamp로 변환
ts_dates = pd.to_datetime(dates)
print(ts_dates)
print('\n')

#timestamp를 period로 변환
pr_day = ts_dates.to_period(freq='D')
print(pr_day)
pr_month = ts_dates.to_period(freq='M')
print(pr_month)
pr_year = ts_dates.to_period(freq='A')
print(pr_year)

DatetimeIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='datetime64[ns]', freq=None)


PeriodIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='period[D]')
PeriodIndex(['2019-01', '2020-03', '2021-06'], dtype='period[M]')
PeriodIndex(['2019', '2020', '2021'], dtype='period[A-DEC]')


freq 옵션 종류
1. D, M, A : 앞에 설명
2. W: 1주 기준
3. MS: 월초 기준
4. Q, QS: 분기말, 분기초 기준
5. AS: 연초 기준
6. B: 휴일 제외
7. H, T, S, L, U, N: 1시간, 1분, 1초, 1/1000초, 1/1000000초, 1/1,000,000,000초

### 시계열 데이터 만들기

Timestamp 배열: 판다스 date_range() 함수를 사용하면 여러 개의 날짜가 들어 있는 배열 형태의 시계열 데이터를 만들 수 있다. 파이썬 range() 햄수로 숫자 배열을 만드는 것과 비슷하다.

In [9]:
# Timestamp의 배열 만들기 - 월 간격, 월의 시작일 기준
ts_ms = pd.date_range(start = '2022-01-01', #날짜 범위 시작
                      end = None,           #날짜 범위 끝
                      periods=6,            #생성할 Timestamp 개수
                      freq='MS',            #시간 간격(MS: 월의 시작일)
                      tz='Asia/Seoul')      #시간대(timezone)
print(ts_ms)

DatetimeIndex(['2022-01-01 00:00:00+09:00', '2022-02-01 00:00:00+09:00',
               '2022-03-01 00:00:00+09:00', '2022-04-01 00:00:00+09:00',
               '2022-05-01 00:00:00+09:00', '2022-06-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='MS')


In [10]:
# Timestamp의 배열 만들기 - 월 간격, 월의 시작일 기준
ts_ms = pd.date_range(start='2019-01-01',    # 날짜 범위의 시작
                   end=None,                 # 날짜 범위의 끝
                   periods=6,                # 생성할 Timestamp의 개수
                   freq='MS',                # 시간 간격 (MS: 월의 시작일)
                   tz='Asia/Seoul')          # 시간대(timezone)
print(ts_ms)
print('\n')

# 월 간격, 월의 마지막 날 기준
ts_me = pd.date_range('2019-01-01', periods=6, 
                   freq='M',              # 시간 간격 (M: 월의 마지막 날)
                   tz='Asia/Seoul')       # 시간대(timezone)
print(ts_me)
print('\n')

# 분기(3개월) 간격, 월의 마지막 날 기준
ts_3m = pd.date_range('2019-01-01', periods=6, 
                   freq='3M',             # 시간 간격 (3M: 3개월)
                   tz='Asia/Seoul')       # 시간대(timezone)
print(ts_3m)


DatetimeIndex(['2019-01-01 00:00:00+09:00', '2019-02-01 00:00:00+09:00',
               '2019-03-01 00:00:00+09:00', '2019-04-01 00:00:00+09:00',
               '2019-05-01 00:00:00+09:00', '2019-06-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='MS')


DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-02-28 00:00:00+09:00',
               '2019-03-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-05-31 00:00:00+09:00', '2019-06-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='M')


DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-07-31 00:00:00+09:00', '2019-10-31 00:00:00+09:00',
               '2020-01-31 00:00:00+09:00', '2020-04-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='3M')


### period 배열

- period_range() 함수는 여러 개의 기간이 들어있는 시계열 데이터를 만든다.
- 결과값에서 Periodindex의 원소는 해당 기간의 전체기간을 나타낸다.

In [11]:
pr_m = pd.period_range(start='2022-01-01',
                       end=None,
                       periods=3,
                       freq='M')
print(pr_m)

PeriodIndex(['2022-01', '2022-02', '2022-03'], dtype='period[M]')


ㅡfreq옵션을 바꾸면서 기간을 다르게 설정해보자.

In [17]:
pr_h = pd.period_range(start='2022-01-01',
                       end=None,
                       periods=3,
                       freq='H')# 1시간 간격
print(pr_h)

PeriodIndex(['2022-01-01 00:00', '2022-01-01 01:00', '2022-01-01 02:00'], dtype='period[H]')


In [18]:
pr_h = pd.period_range(start='2022-01-01',
                       end=None,
                       periods=3,
                       freq='2H') #2시간 간격
print(pr_h)

PeriodIndex(['2022-01-01 00:00', '2022-01-01 02:00', '2022-01-01 04:00'], dtype='period[2H]')


## 시계열 데이터 활용

### 날짜 데이터 분리

연-월-일 날짜 데이터에서 일부를 분리하여 추출할 수 있다. Timestamp로 변환한 객체를 **dt** 속성을 활용하여 year, month, day로 분리가 가능하다.

In [20]:
df = pd.read_csv('./stock-data.csv')

# Date 열을 Timestamp 객체로 변환
df['New_Date'] = pd.to_datetime(df['Date'])
print(df.head())

# dt 속성을 활용해서 연,월,일로 구분
df['Year'] = df['New_Date'].dt.year
df['Month'] = df['New_Date'].dt.month
df['Day'] = df['New_Date'].dt.day

print(df.head())

         Date  Close  Start   High    Low  Volume   New_Date
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26
         Date  Close  Start   High    Low  Volume   New_Date  Year  Month  Day
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02  2018      7    2
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29  2018      6   29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28  2018      6   28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27  2018      6   27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26  2018      6   26


Timestamp 객체를 Period 객체로 변환하는 to_period() 메소드를 적용하여, 연-월-일 중에서 연-월 또는 연도를 추출이 가능하다.

In [21]:
df['Date_yr'] = df['New_Date'].dt.to_period(freq='A')
df['Date_m'] = df['New_Date'].dt.to_period(freq='M')
print(df.head())


         Date  Close  Start   High    Low  Volume   New_Date  Year  Month  \
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02  2018      7   
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29  2018      6   
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28  2018      6   
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27  2018      6   
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26  2018      6   

   Day Date_yr   Date_m  
0    2    2018  2018-07  
1   29    2018  2018-06  
2   28    2018  2018-06  
3   27    2018  2018-06  
4   26    2018  2018-06  


### 날짜 인덱스 활용

- Timestamp로 구성된 열을 행 인덱스로 지정하면 DatetimeIndex라는 고유 속성으로 변환된다. 마찬가지로 Period로 구성된 열을 행 인덱스로 지정하면 PeriodIndex라는 속성을 갖는다.
- 날짜 인덱스를 활용하면 시계열 데이터에 대한 인덱스과 슬라이싱이 편리하다.
- 연-월-일 중에서 내가 필요로 하는 레벨을 선택적으로 인덱싱을 할 수 있다.(연도로만 하던가, 연-월, 연월일 기준으로 선택할 수 있다.)
- 날짜 범위로 슬라이싱도 가능하다.

In [30]:
# read_csv() 함수로 파일 읽어와서 df로 변환
df = pd.read_csv('./stock-data.csv')

# 문자열인 날짜 데이터를 판다스 Timestamp로 변환
df['new_Date'] = pd.to_datetime(df['Date'])   # 새로운 열에 추가
df.sort_values('new_Date')
df.set_index('new_Date', inplace=True)        # 행 인덱스로 지정


print(df.head())
print('\n')
print(df.index)
print('\n')

                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977
2018-06-29  2018-06-29  10700  10550  10900   9990  170253
2018-06-28  2018-06-28  10400  10900  10950  10150  155769
2018-06-27  2018-06-27  10900  10800  11050  10500  133548
2018-06-26  2018-06-26  10800  10900  11000  10700   63039


DatetimeIndex(['2018-07-02', '2018-06-29', '2018-06-28', '2018-06-27',
               '2018-06-26', '2018-06-25', '2018-06-22', '2018-06-21',
               '2018-06-20', '2018-06-19', '2018-06-18', '2018-06-15',
               '2018-06-14', '2018-06-12', '2018-06-11', '2018-06-08',
               '2018-06-07', '2018-06-05', '2018-06-04', '2018-06-01'],
              dtype='datetime64[ns]', name='new_Date', freq=None)




In [37]:
# 날짜 인덱스를 이용하여 데이터 선택하기
df_y = df['2018']
print(df_y.head())
print('\n')
df_ym = df.loc['2018-07']    # loc 인덱서 활용
print(df_ym)
print('\n')
df_ym_cols = df.loc['2018-07', 'Start':'High']    # 열 범위 슬라이싱
print(df_ym_cols)
print('\n')
df_ymd = df.loc['2018-07-02']
print(df_ymd)
print('\n')
df_ymd_range = df['2018-06-20':'2018-06-25']    # 날짜 범위 지정(날짜를 오름차순으로 슬라이싱해야한다.)
print(df_ymd_range)
print('\n')

                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977
2018-06-29  2018-06-29  10700  10550  10900   9990  170253
2018-06-28  2018-06-28  10400  10900  10950  10150  155769
2018-06-27  2018-06-27  10900  10800  11050  10500  133548
2018-06-26  2018-06-26  10800  10900  11000  10700   63039


                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977


            Start   High
new_Date                
2018-07-02  10850  10900


Date      2018-07-02
Close          10100
Start          10850
High           10900
Low            10000
Volume        137977
Name: 2018-07-02 00:00:00, dtype: object


                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-06-25  2018-06-25  11150  11400  1145

Timestamp 객체로 표시된 두 날짜 사이의 시간 간격을 구할 수 있다. 어떤 날짜로 부터 경과한 날짜를 계산하여 행 인덱스를 지정하고, 데이터를 선택하는 과정을 살펴보자.

In [41]:
# 시간 간격 계산. 최근 180일 ~ 189일 사이의 값들만 선택하기
today = pd.to_datetime('2018-12-25')            # 기준일 생성
df['time_delta'] = today - df.index             # 날짜 차이 계산 형태는 XX days로 나온다.
df.set_index('time_delta', inplace=True)        # 행 인덱스로 지정
df_180 = df['180 days':'189 days']
print(df_180)

                  Date  Close  Start   High    Low  Volume
time_delta                                                
180 days    2018-06-28  10400  10900  10950  10150  155769
181 days    2018-06-27  10900  10800  11050  10500  133548
182 days    2018-06-26  10800  10900  11000  10700   63039
183 days    2018-06-25  11150  11400  11450  11000   55519
186 days    2018-06-22  11300  11250  11450  10750  134805
187 days    2018-06-21  11200  11350  11750  11200  133002
188 days    2018-06-20  11550  11200  11600  10900  308596
189 days    2018-06-19  11300  11850  11950  11300  180656
